# Yearly Observations by municipality

Om de regionale verschillen ook in kaart te kunnen brengen, groeperen we niet enkel per jaar, maar ook per gemeente. Dit zijn de 581 hoofdgemeentes in België.

In [2]:
import pandas as pd
import geopandas as gpd

# set the max columns to none
pd.set_option('display.max_columns', None)
# set the max columns to none
pd.set_option('display.max_rows', None)

## Gemeenten in België

In [3]:
## Geodataframe gemeenten in belgië
belgium = gpd.read_file('../resources/geodata/kontur_boundaries_BE_20230628.gpkg')
belgium_gemeenten = belgium[(belgium["osm_admin_level"] == '8')].copy()
belgium_gemeenten = belgium_gemeenten.to_crs(epsg=3857) # for area calculation
belgium_gemeenten["area_km2"]= belgium_gemeenten.area/1_000_000
belgium_gemeenten = belgium_gemeenten.to_crs(epsg=4326) # to match with coördinates of observations

belgium_gemeenten.head(5)
belgium_gemeenten[["name","area_km2"]].head()

,name,area_km2
55,Tenneville,223.330563
56,Rendeux,168.923209
57,Boechout,52.684436
58,Ville de Bruxelles - Stad Brussel,82.945995
59,Spiere-Helkijn,26.958440


In [4]:
belgium_gemeenten['name'].nunique()

581

## Load clean or gold data

In [5]:
yearly = f'../2_cleaning/clean_data/observations_yearly_clean.parquet'
boomklever = f'../3_transformation/gold/observations_bk.parquet'
halsbandparkiet = f'../3_transformation/gold/observations_hp.parquet'

df_yearly_birds = pd.read_parquet(yearly, engine="pyarrow")
df_observations_bk = pd.read_parquet(boomklever, engine="pyarrow")
df_observations_hp = pd.read_parquet(halsbandparkiet, engine="pyarrow")

## Load and transform clean data

In [6]:
# Maak dataframe met alle combinaties van gemeente en periode 'name' and 'year'
all_names = belgium_gemeenten['name'].unique()
all_years = df_yearly_birds.index.unique()
all_combinations = pd.MultiIndex.from_product([all_names, all_years], names=["name", "year"]).to_frame(index=False)

def group_by_year_and_municipality_and_calculate_fields(df_observations):
    df_observations["year"] = df_observations["date"].dt.year
    geometry = gpd.points_from_xy(df_observations['longitude'], df_observations['latitude'])
    gpd_observations = gpd.GeoDataFrame(df_observations, geometry=geometry, crs="EPSG:4326")
    
    # koppel waarnemingen aan gemeenten
    gpd_observations = gpd.sjoin(gpd_observations, belgium_gemeenten, how="right", predicate="within")
    
    # aantal waarnemingen per gemeente en jaar
    result = gpd_observations.groupby(["name", "year"]).agg({'observation_id': 'nunique', 'observer_id': 'nunique', 'area_km2': 'mean'}).rename(columns={'observation_id': 'observation_count', 'observer_id': 'observers_count', 'area_km2':'area_km2'}).reset_index()
    result = all_combinations.merge(result, on=["name", "year"], how='left') # check that all combinations are present
    result.fillna(0, inplace=True) # fill NaN values with 0
    
    # Aandeel per jaarlijks miljoen vogelwaarnemingen
    result = result.merge(df_yearly_birds, on='year', how='left')
    result['observations_pym'] = result['observation_count'] * 1_000_000 / result['allbirds_observation_count'] 
    
    # 5 jaarlijks gemiddelde (fluctuaties opvangen)
    result['observations_pym_5yr_avg'] = result.sort_values('year').groupby(['name'])['observations_pym'].transform(lambda x: x.rolling(window=5, min_periods=1).mean())
    # % groei van 5 jaarlijks gemiddelde over 5 jaar
    result['observations_growth_5yr_%'] = result.sort_values('year').groupby(['name'])['observations_pym_5yr_avg'].transform(lambda x: x.pct_change(periods=5) * 100)
    # Observers per km2
    result['observers_per_km2'] = result['observers_count'] / result['area_km2']
    # Observations per km2
    result['observations_per_km2'] = result['observation_count'] / result['area_km2']
    
    result.drop(columns=['allbirds_observation_count','area_km2'], inplace=True) # not necessary for each species
    return result
    
result_hp = group_by_year_and_municipality_and_calculate_fields(df_observations_hp)
result_bk = group_by_year_and_municipality_and_calculate_fields(df_observations_bk)

# merge the species dataframes
yearly_by_municipal = pd.merge(result_hp, result_bk, on=['year', 'name'], how='outer', suffixes=("_hp", "_bk"))
yearly_by_municipal = yearly_by_municipal.merge(belgium_gemeenten[["name","area_km2"]], on=["name"], how='outer')
yearly_by_municipal = yearly_by_municipal.merge(df_yearly_birds, on='year', how='left') # add allbirds_observation_count



In [7]:
yearly_by_municipal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31374 entries, 0 to 31373
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          31374 non-null  object 
 1   year                          31374 non-null  int64  
 2   observation_count_hp          31374 non-null  float64
 3   observers_count_hp            31374 non-null  float64
 4   observations_pym_hp           31374 non-null  float64
 5   observations_pym_5yr_avg_hp   31374 non-null  float64
 6   observations_growth_5yr_%_hp  6730 non-null   float64
 7   observers_per_km2_hp          3620 non-null   float64
 8   observations_per_km2_hp       3620 non-null   float64
 9   observation_count_bk          31374 non-null  float64
 10  observers_count_bk            31374 non-null  float64
 11  observations_pym_bk           31374 non-null  float64
 12  observations_pym_5yr_avg_bk   31374 non-null  float64
 13  o

In [8]:
# replace NaN values with 0 (eg. growth will be NaN if there are no observations)
yearly_by_municipal.fillna(0, inplace=True) 

# set max and min values for growth (-1000% to 1000%) to avoid outliers and infinity when coming from 0 observations. 
yearly_by_municipal['observations_growth_5yr_%_hp'] = yearly_by_municipal['observations_growth_5yr_%_hp'].clip(lower=-1000, upper=1000)
yearly_by_municipal['observations_growth_5yr_%_bk'] = yearly_by_municipal['observations_growth_5yr_%_bk'].clip(lower=-1000, upper=1000)

yearly_by_municipal.head(10)

,name,year,observation_count_hp,observers_count_hp,observations_pym_hp,observations_pym_5yr_avg_hp,observations_growth_5yr_%_hp,observers_per_km2_hp,observations_per_km2_hp,observation_count_bk,observers_count_bk,observations_pym_bk,observations_pym_5yr_avg_bk,observations_growth_5yr_%_bk,observers_per_km2_bk,observations_per_km2_bk,area_km2,allbirds_observation_count
0,Aalst,1971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,2242
1,Aalst,1972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,5281
2,Aalst,1973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,6547
3,Aalst,1974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,9548
4,Aalst,1975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,9115
5,Aalst,1976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,7035
6,Aalst,1977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,7394
7,Aalst,1978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,11301
8,Aalst,1979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,15202
9,Aalst,1980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,23498


In [9]:
yearly_by_municipal[yearly_by_municipal.isna().any(axis=1)].sort_values(["year"], ascending=False).head(5)

,name,year,observation_count_hp,observers_count_hp,observations_pym_hp,observations_pym_5yr_avg_hp,observations_growth_5yr_%_hp,observers_per_km2_hp,observations_per_km2_hp,observation_count_bk,observers_count_bk,observations_pym_bk,observations_pym_5yr_avg_bk,observations_growth_5yr_%_bk,observers_per_km2_bk,observations_per_km2_bk,area_km2,allbirds_observation_count


## Write result to parquet-file in "gold" folder

In [10]:
yearly_by_municipal.to_parquet(f'./gold/yearly_observations_by_municipality.parquet', engine="pyarrow")